# Comparsion of "pyspark" and "pandas"

* **Language  : Python**
* **Dataset   : Kaggle Titanic -** https://www.kaggle.com/c/titanic/data?select=train.csv 
* **Author    : singhmnprt01**   

In [658]:
import matplotlib
import seaborn
import plotly
import pandas as pd
import numpy as np
import pickle
import warnings
import pyspark
import os
#from pyspark import SparkContext, SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc, asc, col, when, expr, mean, pandas_udf,udf
import pyspark.sql.functions as f
from pyspark.sql.types import *

warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
pyspark.__version__

'3.0.1'

In [ ]:
###  IMPORTANT: to make worker and driver use same python version ###

os.environ['PYSPARK_PYTHON'] = '/Users/manpreetsi/Applications/anaconda3/envs/python_36/bin/python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/Users/manpreetsi/Applications/anaconda3/envs/python_36/bin/python'

In [691]:
spark = (SparkSession.builder.appName("spark_practise").getOrCreate())

In [4]:
pd.set_option("display.max_columns",None)

In [61]:
os.getcwd()

'/Users/manpreetsi/My Python Jupyter Notebook/Learn_Python'

In [62]:
os.chdir('/Users/manpreetsi/My Python Jupyter Notebook')

## 1 Read a CSV file  

### pandas

In [505]:
data_pandas = pd.read_csv("train.csv")


In [64]:
type(data_pandas)

pandas.core.frame.DataFrame

In [65]:
data_pandas.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [67]:
data_pandas['Name'].head(n=10)

0                              Braund, Mr. Owen Harris
1    Cumings, Mrs. John Bradley (Florence Briggs Th...
2                               Heikkinen, Miss. Laina
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)
4                             Allen, Mr. William Henry
5                                     Moran, Mr. James
6                              McCarthy, Mr. Timothy J
7                       Palsson, Master. Gosta Leonard
8    Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)
9                  Nasser, Mrs. Nicholas (Adele Achem)
Name: Name, dtype: object

In [74]:
data_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


### spark

In [692]:
schema =  StructType([StructField("PassengerId",IntegerType(),False),
                     StructField("Survived",IntegerType(),False),
                     StructField("Pclass",IntegerType(),False),
                     StructField("Name",StringType(),False),
                     StructField("Sex",StringType(),False),
                     StructField("Age",FloatType(),False),
                     StructField("SibSp",IntegerType(),False),
                     StructField("Parch",IntegerType(),False),
                     StructField("Ticket",StringType(),False),
                     StructField("Fare",FloatType(),False),
                     StructField("Cabin",StringType(),False),
                     StructField("Embarked",StringType(),False)])

In [693]:
data_spark = spark.read.format("csv").option("header","true").schema(schema).load("/Users/manpreetsi/My Python Jupyter Notebook/train.csv")


In [694]:
type(data_spark)

pyspark.sql.dataframe.DataFrame

In [132]:
data_spark.show(n=2)

+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|   Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0| PC 17599|71.2833|  C85|       C|
+-----------+--------+------+--------------------+------+----+-----+-----+---------+-------+-----+--------+
only showing top 2 rows



In [133]:
data_spark.select(['Name']).show(n=10)

+--------------------+
|                Name|
+--------------------+
|Braund, Mr. Owen ...|
|Cumings, Mrs. Joh...|
|Heikkinen, Miss. ...|
|Futrelle, Mrs. Ja...|
|Allen, Mr. Willia...|
|    Moran, Mr. James|
|McCarthy, Mr. Tim...|
|Palsson, Master. ...|
|Johnson, Mrs. Osc...|
|Nasser, Mrs. Nich...|
+--------------------+
only showing top 10 rows



In [134]:
data_spark.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: float (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## 2 Sort DataFrame based on a column

### pandas

In [135]:
data_pandas.sort_values(by='Age', ascending=False)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
630,631,1,1,"Barkworth, Mr. Algernon Henry Wilson",male,80.0,0,0,27042,30.0000,A23,S
851,852,0,3,"Svensson, Mr. Johan",male,74.0,0,0,347060,7.7750,NaN,S
493,494,0,1,"Artagaveytia, Mr. Ramon",male,71.0,0,0,PC 17609,49.5042,NaN,C
96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,PC 17754,34.6542,A5,C
116,117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,370369,7.7500,NaN,Q
...,...,...,...,...,...,...,...,...,...,...,...,...
859,860,0,3,"Razi, Mr. Raihed",male,NaN,0,0,2629,7.2292,NaN,C
863,864,0,3,"Sage, Miss. Dorothy Edith ""Dolly""",female,NaN,8,2,CA. 2343,69.5500,NaN,S
868,869,0,3,"van Melkebeke, Mr. Philemon",male,NaN,0,0,345777,9.5000,NaN,S
878,879,0,3,"Laleff, Mr. Kristo",male,NaN,0,0,349217,7.8958,NaN,S


### spark

In [296]:
data_spark.sort(col("Age").desc()).show() ## using col function

+-----------+--------+------+--------------------+------+-----+-----+-----+-----------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|     Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+-----------+-------+-----------+--------+
|        631|       1|     1|Barkworth, Mr. Al...|  male|160.0|    0|    0|      27042|   30.0|        A23|       S|
|        852|       0|     3| Svensson, Mr. Johan|  male|148.0|    0|    0|     347060|  7.775|       null|       S|
|         97|       0|     1|Goldschmidt, Mr. ...|  male|142.0|    0|    0|   PC 17754|34.6542|         A5|       C|
|        494|       0|     1|Artagaveytia, Mr....|  male|142.0|    0|    0|   PC 17609|49.5042|       null|       C|
|        117|       0|     3|Connors, Mr. Patrick|  male|141.0|    0|    0|     370369|   7.75|       null|       Q|
|        746|       0|     1|Crosby, Capt. Edw...|  male|140.0| 

In [137]:
data_spark.sort(data_spark.Age.desc()).show()

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+-----------+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|   Fare|      Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+-----------+--------+
|        631|       1|     1|Barkworth, Mr. Al...|  male|80.0|    0|    0|      27042|   30.0|        A23|       S|
|        852|       0|     3| Svensson, Mr. Johan|  male|74.0|    0|    0|     347060|  7.775|       null|       S|
|         97|       0|     1|Goldschmidt, Mr. ...|  male|71.0|    0|    0|   PC 17754|34.6542|         A5|       C|
|        494|       0|     1|Artagaveytia, Mr....|  male|71.0|    0|    0|   PC 17609|49.5042|       null|       C|
|        117|       0|     3|Connors, Mr. Patrick|  male|70.5|    0|    0|     370369|   7.75|       null|       Q|
|        746|       0|     1|Crosby, Capt. Edw...|  male|70.0|    1|    

In [147]:
data_spark.orderBy('Age', ascending=False).show(10)

+-----------+--------+------+--------------------+----+----+-----+-----+----------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|    Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+----------+-------+-----+--------+
|        631|       1|     1|Barkworth, Mr. Al...|male|80.0|    0|    0|     27042|   30.0|  A23|       S|
|        852|       0|     3| Svensson, Mr. Johan|male|74.0|    0|    0|    347060|  7.775| null|       S|
|         97|       0|     1|Goldschmidt, Mr. ...|male|71.0|    0|    0|  PC 17754|34.6542|   A5|       C|
|        494|       0|     1|Artagaveytia, Mr....|male|71.0|    0|    0|  PC 17609|49.5042| null|       C|
|        117|       0|     3|Connors, Mr. Patrick|male|70.5|    0|    0|    370369|   7.75| null|       Q|
|        746|       0|     1|Crosby, Capt. Edw...|male|70.0|    1|    1| WE/P 5735|   71.0|  B22|       S|
|        673|       0|     2|Mitchell

In [149]:
## sorting based on more than 1 column 


data_spark.orderBy(['Age','Fare'], ascending=[False,True]).show(10) 

+-----------+--------+------+--------------------+----+----+-----+-----+----------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name| Sex| Age|SibSp|Parch|    Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+----+----+-----+-----+----------+-------+-----+--------+
|        631|       1|     1|Barkworth, Mr. Al...|male|80.0|    0|    0|     27042|   30.0|  A23|       S|
|        852|       0|     3| Svensson, Mr. Johan|male|74.0|    0|    0|    347060|  7.775| null|       S|
|         97|       0|     1|Goldschmidt, Mr. ...|male|71.0|    0|    0|  PC 17754|34.6542|   A5|       C|
|        494|       0|     1|Artagaveytia, Mr....|male|71.0|    0|    0|  PC 17609|49.5042| null|       C|
|        117|       0|     3|Connors, Mr. Patrick|male|70.5|    0|    0|    370369|   7.75| null|       Q|
|        673|       0|     2|Mitchell, Mr. Hen...|male|70.0|    0|    0|C.A. 24580|   10.5| null|       S|
|        746|       0|     1|Crosby, 

## 3 Where condition on a column - Filter

### pandas

In [144]:
data_pandas[data_pandas.Age>50].head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
11,12,1,1,"Bonnell, Miss. Elizabeth",female,58.0,0,0,113783,26.5500,C103,S
15,16,1,2,"Hewlett, Mrs. (Mary D Kingcome)",female,55.0,0,0,248706,16.0000,NaN,S
33,34,0,2,"Wheadon, Mr. Edward H",male,66.0,0,0,C.A. 24579,10.5000,NaN,S
54,55,0,1,"Ostby, Mr. Engelhart Cornelius",male,65.0,0,1,113509,61.9792,B30,C
94,95,0,3,"Coxon, Mr. Daniel",male,59.0,0,0,364500,7.2500,NaN,S
96,97,0,1,"Goldschmidt, Mr. George B",male,71.0,0,0,PC 17754,34.6542,A5,C
116,117,0,3,"Connors, Mr. Patrick",male,70.5,0,0,370369,7.7500,NaN,Q
124,125,0,1,"White, Mr. Percival Wayland",male,54.0,0,1,35281,77.2875,D26,S
150,151,0,2,"Bateman, Rev. Robert James",male,51.0,0,0,S.O.P. 1166,12.5250,NaN,S


### spark - where

In [145]:
data_spark.where(data_spark.Age>50).show(10)

+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|     Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+-----------+-------+-----+--------+
|          7|       0|     1|McCarthy, Mr. Tim...|  male|54.0|    0|    0|      17463|51.8625|  E46|       S|
|         12|       1|     1|Bonnell, Miss. El...|female|58.0|    0|    0|     113783|  26.55| C103|       S|
|         16|       1|     2|Hewlett, Mrs. (Ma...|female|55.0|    0|    0|     248706|   16.0| null|       S|
|         34|       0|     2|Wheadon, Mr. Edwa...|  male|66.0|    0|    0| C.A. 24579|   10.5| null|       S|
|         55|       0|     1|Ostby, Mr. Engelh...|  male|65.0|    0|    1|     113509|61.9792|  B30|       C|
|         95|       0|     3|   Coxon, Mr. Daniel|  male|59.0|    0|    0|     364500|   7.25| null|       S|
|         

### spark - filter

In [206]:
data_spark.filter(data_spark.Age>50).show(10)

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          2|       1|     1|Cumings, Mrs. Joh...|female| 76.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 52.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 70.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 70.0|    0|    0|          373450|   8.05| null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|108.0|    0|    0|           17463|51.8625|  E46|       S|
|          9|       1|     3|Johnson, Mrs. Osc...|female| 54.0|    0|   

## 4 Filter on a column and select limited columns + sort the data

### pandas

In [151]:
data_pandas[["PassengerId","Name","Age"]][data_pandas.Age>50].sort_values(by='Age',ascending=False).head(10)

,PassengerId,Name,Age
630,631,"Barkworth, Mr. Algernon Henry Wilson",80.0
851,852,"Svensson, Mr. Johan",74.0
493,494,"Artagaveytia, Mr. Ramon",71.0
96,97,"Goldschmidt, Mr. George B",71.0
116,117,"Connors, Mr. Patrick",70.5
672,673,"Mitchell, Mr. Henry Michael",70.0
745,746,"Crosby, Capt. Edward Gifford",70.0
33,34,"Wheadon, Mr. Edward H",66.0
54,55,"Ostby, Mr. Engelhart Cornelius",65.0
456,457,"Millet, Mr. Francis Davis",65.0


### spark

In [155]:
data_spark.select(['PassengerId','Name','Age']).where(data_spark.Age>50).orderBy('Age',ascending=False).show(10)

+-----------+--------------------+----+
|PassengerId|                Name| Age|
+-----------+--------------------+----+
|        631|Barkworth, Mr. Al...|80.0|
|        852| Svensson, Mr. Johan|74.0|
|         97|Goldschmidt, Mr. ...|71.0|
|        494|Artagaveytia, Mr....|71.0|
|        117|Connors, Mr. Patrick|70.5|
|        746|Crosby, Capt. Edw...|70.0|
|        673|Mitchell, Mr. Hen...|70.0|
|         34|Wheadon, Mr. Edwa...|66.0|
|         55|Ostby, Mr. Engelh...|65.0|
|        281|    Duane, Mr. Frank|65.0|
+-----------+--------------------+----+
only showing top 10 rows



## 5 if_else operation on a column

### pandas

In [425]:
data_pandas['Age']= data_pandas['Age'].apply(lambda x : x*2 if x>45 else x)

In [426]:
data_pandas.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,108.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [202]:
data_pandas['Age'].apply(lambda x : x*2 if x>50 else x).head(10)

0     44.0
1    152.0
2    104.0
3    140.0
4    140.0
5      NaN
6    216.0
7      4.0
8    108.0
9     28.0
Name: Age, dtype: float64

In [ ]:
'''
PySpark withColumn() is a transformation function of DataFrame which is used to 
1. change or update the value, 
2. convert the datatype of an existing DataFrame column, 
3. add/create a new column, and many-core

lit() function is used to add a constant value to a DataFrame column.
'''

In [197]:

##data_spark = data_spark.withColumn('Age',data_spark.Age*2)
##or 

data_spark = data_spark.withColumn('Age',col('Age')*2)

In [198]:
data_spark.show(10)
#data_spark.withColumn('Age',col('Age')*2).show()

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 44.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 76.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 52.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 70.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 70.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male| null|    0|   

In [200]:
##data_spark.withColumn('Age', when(data_spark.Age>50,data_spark.Age*2).otherwise(data_spark.Age)).show()
## or
data_spark.withColumn('Age', when(data_spark.Age>50,col('Age')*2).otherwise(col('Age'))).show(10)

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 44.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|152.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|104.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|140.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|140.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male| null|    0|   

## 6  isin function

### pandas

In [212]:
## This will give error as pandas isin expects a list like object
data_pandas[data_pandas.Embarked.isin('S')] 

TypeError: only list-like objects are allowed to be passed to isin(), you passed a [str]

In [214]:
data_pandas[data_pandas.Embarked.isin(['S'])]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,44.0,1,0,A/5 21171,7.2500,NaN,S
2,3,1,3,"Heikkinen, Miss. Laina",female,52.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,70.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,70.0,0,0,373450,8.0500,NaN,S
6,7,0,1,"McCarthy, Mr. Timothy J",male,108.0,0,0,17463,51.8625,E46,S
...,...,...,...,...,...,...,...,...,...,...,...,...
883,884,0,2,"Banfield, Mr. Frederick James",male,56.0,0,0,C.A./SOTON 34068,10.5000,NaN,S
884,885,0,3,"Sutehall, Mr. Henry Jr",male,50.0,0,0,SOTON/OQ 392076,7.0500,NaN,S
886,887,0,2,"Montvila, Rev. Juozas",male,54.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,38.0,0,0,112053,30.0000,B42,S


### spark

In [306]:
data_spark.where(data_spark.Embarked.isin(['S','C'])).show()

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 44.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 76.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 52.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 70.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 70.0|    0|    0|          373450|   8.05| null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|108.0|    0|   

In [219]:
data_spark.filter(data_spark.Embarked.isin(['S','C'])).show()

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 44.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 76.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 52.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 70.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 70.0|    0|    0|          373450|   8.05| null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|108.0|    0|   

In [220]:
## without list like object
data_spark.filter(data_spark.Embarked.isin('S','C')).show()

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 44.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 76.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 52.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 70.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 70.0|    0|    0|          373450|   8.05| null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|108.0|    0|   

In [221]:
## using col
data_spark.filter(col('Embarked').isin('S','C')).show()

+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex|  Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+-----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male| 44.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female| 76.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female| 52.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female| 70.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male| 70.0|    0|    0|          373450|   8.05| null|       S|
|          7|       0|     1|McCarthy, Mr. Tim...|  male|108.0|    0|   

## 7 Aggregations - count,sum, min, max, avg, mean, median + groupby & orderby 

### Sum

#### pandas

In [224]:
sum(data_pandas['Age'][~data_pandas.Age.isna()])

42410.34

#### spark

In [249]:
data_spark.select(['Age']).where(data_spark.Age.isNotNull()).groupBy().sum().show()

+------------------+
|          sum(Age)|
+------------------+
|42410.339999973774|
+------------------+



###  Columns Selection + Sum + Filter_Null + Groupby + Orderby

* **Query: Get Sum Age for each Embarked**

### pandas

In [281]:
data_pandas[(~data_pandas.Age.isna()) & (~data_pandas.Embarked.isna())]\
.groupby(['Embarked'])['Embarked','Age']\
.sum()\
.sort_values(by='Embarked',ascending=False)



,Age
Embarked,
S,32625.50
Q,1573.00
C,8011.84


### Spark

In [356]:
data_spark\
.select(['Embarked','Age'])\
.where(data_spark.Age.isNotNull() & data_spark.Embarked.isNotNull())\
.groupBy('Embarked')\
.sum('Age')\
.orderBy("Embarked", ascending=False)\
.show() 

+--------+-----------------+
|Embarked|         sum(Age)|
+--------+-----------------+
|       S|          32625.5|
|       Q|           1573.0|
|       C|8011.839999973774|
+--------+-----------------+



* **Query: Get min, max and sum age for each Embarked**

In [358]:
data_spark\
.select('Embarked','Age')\
.where(data_spark.Embarked.isNotNull() & data_spark.Age.isNotNull())\
.groupBy('Embarked')\
.agg(f.min(col('Age')).alias('min_age'), f.max(col('Age')).alias('max_age'), f.sum(col('Age')).alias('sum_age'))\
.orderBy('min_age')\
.show()

+--------+-------+-------+-----------------+
|Embarked|min_age|max_age|          sum_age|
+--------+-------+-------+-----------------+
|       C|   0.84|  142.0|8011.839999973774|
|       S|   1.34|  160.0|          32625.5|
|       Q|    4.0|  141.0|           1573.0|
+--------+-------+-------+-----------------+



**or**

In [359]:
my_agg=[f.min(col('Age')).alias('min_age'), f.max(col('Age')).alias('max_age'), f.sum(col('Age')).alias('sum_age')]

In [363]:
data_spark\
.select('Embarked','Age')\
.where(data_spark.Embarked.isNotNull() & data_spark.Age.isNotNull())\
.groupBy('Embarked')\
.agg(*my_agg)\
.orderBy('min_age')\
.show()

+--------+-------+-------+-----------------+
|Embarked|min_age|max_age|          sum_age|
+--------+-------+-------+-----------------+
|       C|   0.84|  142.0|8011.839999973774|
|       S|   1.34|  160.0|          32625.5|
|       Q|    4.0|  141.0|           1573.0|
+--------+-------+-------+-----------------+



* **Query: Get distinct Embarked values** 

In [351]:
data_spark.select('Embarked').where(data_spark.Embarked.isNotNull()).distinct().show()

+--------+
|Embarked|
+--------+
|       Q|
|       C|
|       S|
+--------+



* **Query: Get count per embarked**

In [429]:
data_spark\
.select('Embarked')\
.where(data_spark.Embarked.isNotNull())\
.groupBy('Embarked')\
.agg(f.count('Embarked').alias('Count_Embarked'))\
.orderBy('Embarked')\
.show()


+--------+--------------+
|Embarked|Count_Embarked|
+--------+--------------+
|       C|           168|
|       Q|            77|
|       S|           644|
+--------+--------------+



* **Query: Average age of female candidate per Embarked who survived**

In [ ]:
## This gives error as Age filter is float, and other tow are boolean
'''
data_spark.\
select('Embarked','Age')\
.where(data_spark.Age>40 & data_spark.Embarked.isNotNull() & data_spark.Sex.isin('female')& data_spark.Survived.isin(1))\
.groupBy('Embarked')\
.agg(f.sum('Age'))\
.show()

In [405]:
data_spark.\
select('Embarked','Age')\
.where(data_spark.Age>40)\
.where(data_spark.Embarked.isNotNull() & data_spark.Sex.isin('female') & data_spark.Survived.isin(1))\
.groupBy('Embarked')\
.agg(f.sum('Age'))\
.show()

+--------+--------+
|Embarked|sum(Age)|
+--------+--------+
|       Q|   170.0|
|       C|  2770.0|
|       S|  6943.0|
+--------+--------+



## 8 Change data type of a column

### pandas

In [414]:
data_pandas['PassengerId'] = data_pandas['PassengerId'].astype('object')

In [421]:
data_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    object 
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(4), object(6)
memory usage: 83.7+ KB


### spark

In [419]:
data_spark = data_spark.withColumn('PassengerId', col('PassengerId').cast(StringType()))

In [420]:
data_spark.printSchema()

root
 |-- PassengerId: string (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sex: string (nullable = true)
 |-- Age: float (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: float (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



## 9 Fillna of a column with a value

### pandas

In [508]:
data_pandas['Age'].isnull().any()

True

In [509]:
sum(data_pandas['Age'].isnull())

177

In [510]:
(data_pandas['Age'][~data_pandas.Age.isnull()]).mean()

29.69911764705882

In [531]:
type((data_pandas['Age'][~data_pandas.Age.isnull()]).mean())

numpy.float64

In [514]:
data_pandas['Age'] = data_pandas['Age'].fillna((data_pandas['Age'][~data_pandas.Age.isnull()]).mean())

In [515]:
data_pandas['Age'].isnull().any()

False

In [610]:
sum(data_pandas['Age'].isnull())

0

### spark

In [511]:
data_spark.where(data_spark.Age.isNull()).groupBy('Age').agg(f.count(f.lit(1))).show()

+----+--------+
| Age|count(1)|
+----+--------+
|null|     177|
+----+--------+



**or**

In [512]:
data_spark.where(data_spark.Age.isNull()).count()

177

In [602]:
## collect() returns a list
x = data_spark.where(data_spark.Age.isNotNull()).agg(f.mean('Age').alias("Average_Age")).collect()[0][0]

In [603]:
round(x,2)

29.7

In [604]:
## Mthod 1 
data_spark.fillna(value=round(x,2),subset=['Age']).show(6)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| null|       S|
|          6|       0|     3|    Moran, Mr. James|  male|29.7|    0|    0|      

In [606]:
## Method 2
data_spark.withColumn('Age', when(data_spark.Age.isNull(),f.lit(round(x,2))).otherwise(col('Age')))

In [609]:
## Assigning the action to the dataframe
data_spark = data_spark.withColumn('Age', when(data_spark.Age.isNull(),f.lit(round(x,2))).otherwise(col('Age')))

In [608]:
data_spark.where(data_spark.Age.isNull()).count()

0

In [612]:
data_spark.where(data_spark.Age.isNull()).agg(f.count(f.lit(1))).show()

+--------+
|count(1)|
+--------+
|       0|
+--------+



In [613]:
sum(data_pandas['Age'].isnull())

0

## 10 Joins

### pandas

In [630]:
## left join on PassengerId

pd.merge(data_pandas,data_pandas,on='PassengerId',how='left', suffixes=("_left","_right")).head(4)

,PassengerId,Survived_left,Pclass_left,Name_left,Sex_left,Age_left,SibSp_left,Parch_left,Ticket_left,Fare_left,Cabin_left,Embarked_left,Survived_right,Pclass_right,Name_right,Sex_right,Age_right,SibSp_right,Parch_right,Ticket_right,Fare_right,Cabin_right,Embarked_right
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


### spark

In [637]:
data_spark.join(data_spark,data_spark.PassengerId==data_spark.PassengerId,"left").show(4)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 175

In [638]:
data_spark.join(data_spark,data_spark.PassengerId==data_spark.PassengerId,"leftsemi").show(4)

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|   Sex| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| null|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| null|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
only showing top 4 rows



## 11 List comprehension

* **Query: Get num of null values in each column of the dataframe**

### pandas

In [642]:
[sum(data_pandas[i].isnull()) for i in list(data_pandas.columns)]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 687, 2]

In [643]:
list(data_pandas.columns)

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

### spark

In [650]:
[data_spark.where(col(i).isNull()).count() for i in data_spark.columns]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 687, 2]

In [651]:
data_spark.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

## 12 Pandas UDFs in spark

### Series to Series python type hints

In [762]:
def eligibility_checker(x):
    return pd.Series(["Eligible" if a >= 18 else "Ineligible" for a in x])

In [776]:
spark.udf.register("eligibility_checker",eligibility_checker,StringType())

<function __main__.eligibility_checker(x)>

In [763]:
elig_udf = pandas_udf(eligibility_checker, returnType=StringType())

In [778]:
data_spark.withColumn("Eligibility_check",elig_udf(col('Age'))).select('Age','Eligibility_check').show(5)

+----+-----------------+
| Age|Eligibility_check|
+----+-----------------+
|22.0|         Eligible|
|38.0|         Eligible|
|26.0|         Eligible|
|35.0|         Eligible|
|35.0|         Eligible|
+----+-----------------+
only showing top 5 rows



### Series to Scalar python type hints

In [766]:
def age_double(x):
    return x*2

In [767]:
age_double_udf = pandas_udf(age_double, returnType=FloatType())

In [779]:
data_spark.withColumn('Age_Double',age_double_udf(col('Age'))).select('Age','Age_Double').show(5)

+----+----------+
| Age|Age_Double|
+----+----------+
|22.0|      44.0|
|38.0|      76.0|
|26.0|      52.0|
|35.0|      70.0|
|35.0|      70.0|
+----+----------+
only showing top 5 rows



### Using both UDFs together

In [780]:
data_spark\
.withColumn('Age_Double',age_double_udf(col('Age')))\
.withColumn("Eligibility_check",elig_udf(col('Age')))\
.select('PassengerId','Age','Age_Double','Eligibility_check')\
.show(5)

+-----------+----+----------+-----------------+
|PassengerId| Age|Age_Double|Eligibility_check|
+-----------+----+----------+-----------------+
|          1|22.0|      44.0|         Eligible|
|          2|38.0|      76.0|         Eligible|
|          3|26.0|      52.0|         Eligible|
|          4|35.0|      70.0|         Eligible|
|          5|35.0|      70.0|         Eligible|
+-----------+----+----------+-----------------+
only showing top 5 rows



In [781]:
@pandas_udf('long')
def pandas_plus_one(s) :
    return s + 1



In [782]:
spark.range(10).select(pandas_plus_one("id")).show()

+-------------------+
|pandas_plus_one(id)|
+-------------------+
|                  1|
|                  2|
|                  3|
|                  4|
|                  5|
|                  6|
|                  7|
|                  8|
|                  9|
|                 10|
+-------------------+

